In [13]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [14]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [16]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['ts',
  'IP_flags',
  'IP_DF',
  'TCP_dataofs',
  'TCP_SYN',
  'TCP_ACK',
  'sport_class',
  'dst_IP_diversity',
  'dst_port_diversity',
  'pck_size_sum_of_EW',
  'ts_diff',
  'ts_std_WE',
  'ts_sum_of_EW',
  'TCP_window_std_WE',
  'pck_size_mean_2',
  'ts_mean_2',
  'ts_std_2',
  'TCP_window_mean_2',
  'TCP_SYN_sum',
  'TCP_ACK_sum',
  'TCP_SYN_ratio',
  'TCP_ACK_SR',
  'ts_mean_6',
  'ts_std_6',
  'pck_size_mean_9',
  'ts_mean_9',
  'ts_std_9',
  'TCP_window_mean_9',
  'TCP_ACK_R',
  'Label'],
 'ARP': ['Ether_type',
  'IP_tos',
  'IP_flags',
  'IP_MF',
  'IP_ttl',
  'IP_proto',
  'ICMP_type',
  'TCP_reserved',
  'TCP_ACK',
  'TCP_ECE',
  'TCP_CWR',
  'UDP_len',
  'DNS_qr',
  'DNS_ra',
  'DNS_qdcount',
  'DNS_nscount',
  'DNS_arcount',
  'payload_bytes',
  'entropy',
  'Protocol',
  'dst_IP_diversity',
  'pck_size_diff',
  'pck_size_mean_WE',
  'pck_size_std_WE',
  'pck_size_sum_of_EW',
  'ts_std_WE',
  'TCP_window_mean_WE',
  'TCP_window_std_WE',
  'TCP_window_sum_of_EW',
  

In [11]:
file_list={
    "SYN_ARP":['./train-data/syn-arp-attack-train.csv','./train-data/syn-arp-attack-test.csv'],
# 'SYN': ['./INPUT/SM/DoS-SYN-1.csv', './INPUT/VAL/VAL-SYN.csv'],
#  'HTTP': ['./INPUT/SM/MB-HTTP-4.csv', './INPUT/VAL/VAL-HTTP.csv'],
#  'ACK': ['./INPUT/SM/MB-ACK-4.csv', './INPUT/VAL/VAL-ACK.csv'],
#  'UDP': ['./INPUT/SM/MB-UDP-4.csv', './INPUT/VAL/VAL-UDP.csv'],
#  'ARP': ['./INPUT/SM/MitM-ARP-6.csv', './INPUT/VAL/VAL-ARP.csv'],
#  'SP': ['./INPUT/SM/Scan-Port-3.csv', './INPUT/VAL/VAL-SP.csv'],
#  'BF': ['./INPUT/SM/MB-BF-5.csv', './INPUT/VAL/VAL-BF.csv'],
#  'OS': ['./INPUT/SM/Scan-OS-3.csv', './INPUT/VAL/VAL-OS.csv'],
#  'SCHD': ['./INPUT/SM/Scan-HDis-3.csv', './INPUT/VAL/VAL-SCHD.csv'],
#  'MHDis': ['./INPUT/SM/MB-HDis-3.csv', './INPUT/VAL/VAL-MHDis.csv']
 }

In [6]:
#Thực hiện tìm kiếm ngẫu nhiên (RandomizedSearchCV) để tối ưu hóa siêu tham số của mô hình.
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV RF

In [ ]:
#Tối ưu hóa siêu tham số cho Random Forest
#Mục đích: Tối ưu hóa siêu tham số cho mô hình Random Forest bằng cách sử dụng RandomizedSearchCV và lưu kết quả vào tệp CSV.
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    #  đảm bảo thứ cột giông nhau
    df = df[feature_list[j]]
    print(feature_list[j])
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
     #  đảm bảo thứ cột giông nhau
    df = df[feature_list[j]]
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  

    X= np.concatenate([X_train, X_test])
    # -1 cho tập huán luyện , 0 cho tâp kiểm tra
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold) # phân chia các fold theo test_fold
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

SYN_ARP
['pck_size_mean_6', 'ts_std_9', 'ts_sum_of_EW', 'dst_IP_diversity', 'entropy_std_6', 'TCP_reserved', 'pck_size_sum_of_EW', 'TCP_PSH_ratio', 'TCP_CWR_SR', 'ts_mean_6', 'payload_bytes_std_6', 'ts_diff', 'TCP_window_std_WE', 'ts_std_2', 'TCP_ACK', 'Protocol', 'TCP_ACK_SR', 'TCP_RST_SR', 'TCP_window_mean_WE', 'payload_bytes_mean_6', 'pck_size_std_9', 'DNS_qr', 'entropy', 'ICMP_type', 'sport_class', 'pck_size_std_WE', 'ts_mean_2', 'ts_std_WE', 'TCP_SYN_ratio', 'TCP_ACK_R', 'payload_bytes', 'entropy_diff', 'pck_size_mean_9', 'TCP_CWR', 'TCP_RST_ratio', 'pck_size_mean_2', 'TCP_URG_SR', 'IP_MF', 'entropy_mean_2', 'dst_port_diversity', 'ts_mean_9', 'IP_flags', 'IP_DF', 'DNS_ra', 'pck_size_mean_WE', 'TCP_window_std_6', 'TCP_window_sum_of_EW', 'ts_std_6', 'DNS_nscount', 'entropy_std_9', 'TCP_window_mean_9', 'TCP_dataofs', 'TCP_SYN', 'payload_bytes_mean_9', 'TCP_PSH_SR', 'TCP_CWR_ratio', 'entropy_sum_of_EW', 'TCP_window_mean_2', 'UDP_len', 'IP_tos', 'entropy_std_WE', 'payload_bytes_mean_WE

 20%|██        | 2/10 [03:05<12:20, 92.53s/it]


KeyboardInterrupt: 

# GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    #  Lớp PredefinedSplit sẽ giúp bạn sử dụng sự phân chia này trong các kỹ thuật như cross-validation mà không cần phải làm lại phân chia dữ liệu.
    ps = PredefinedSplit(test_fold) 
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.9933067878021322   0.044    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.12it/s]


HTTP
default                             0.6315732269934049   0.047    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.52it/s]


ACK
default                             0.9105941815426182   0.015    0       


...

SCHD
default                             0.866209454746268    0.031    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.81it/s]


MHDis
default                             0.7956485114074345   0.014    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.88it/s]

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      4.32876e-09  0.992101  0             0.824     0  SYN
 1      8.11131e-09  0.991096  0             0.772     1  SYN
 2      6.57933e-09  0.991498  0             0.916     2  SYN
 3      3.51119e-09  0.992402  0             0.76      3  SYN
 4      3.51119e-09  0.992402  0             0.883     4  SYN
 5      5.3367e-09   0.991749  1.11022e-16   0.92      5  SYN
 6      2.31013e-09  0.992854  0             0.94      6  SYN
 7      2.84804e-07  0.975805  0             0.991     7  SYN
 8      1e-08        0.990694  0             0.944     8  SYN
 9      1.23285e-09  0.993206  1.11022e-16   0.959     9  SYN
10      1.87382e-09  0.631096  0             0.659     0  HTTP
11      1e-08        0.628884  0             0.752     1  HTTP
12      1.87382e-09  0.631096  0             0.689     2  HTTP
13      2.84804e-09  0.630567  0             0.619     3  H

In [13]:

final_parametres=[["var_smoothing","F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      1.23285e-09  0.993206  1.11022e-16   0.959     9  SYN
 1      1e-09        0.631573  0             0.652     5  HTTP
 2      2.84804e-07  1         0             0.302     0  ACK
 3      1.23285e-09  0.67223   0             0.631     6  UDP
 4      1.23285e-08  0.499829  0             1.23      9  ARP
 5      1e-09        0.976647  1.11022e-16   0.207     2  SP
 6      0.053367     0.716004  0             0.409     2  BF
 7      1e-09        0.791791  0             0.67      2  OS
 8      0.00432876   0.935426  0             0.522     2  SCHD
 9      1e-09        0.795649  0             0.347     0  MHDis


-------------

# RandomizedSearchCV  SVM

In [ ]:
# C là tham số điều chỉnh mức độ phạt đối với các lỗi phân loại trong SVM.
# gamma là tham số điều chỉnh phạm vi ảnh hưởng của mỗi điểm dữ liệu trong SVM
# gamma nhỏ sẽ khiến cho sự ảnh hưởng của mỗi điểm dữ liệu rất rộng, trong khi một giá trị gamma lớn sẽ khiến mỗi điểm dữ liệu chỉ ảnh hưởng đến một khu vực nhỏ.

lines=[[ 'gamma','C', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    #  chia dữ liệu như cross-validation mà không cần phải làm lại phân chia dữ liệu.
    ps = PredefinedSplit(test_fold)
    
    param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  

    second=time()
    f1=[]
    clf=svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("svm_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9258933118713963   14.975   0       


100%|████████████████████████████████████████████| 1/1 [08:57<00:00, 537.67s/it]

{'gamma': 1, 'C': 10}                                                                      0.9998489480892209   537.668  0       
HTTP


default                             0.4897829026250669   3.253    0       


100%|████████████████████████████████████████████| 1/1 [10:42<00:00, 642.06s/it]

...

100%|█████████████████████████████████████████████| 1/1 [01:32<00:00, 92.68s/it]

{'gamma': 1, 'C': 10}                                                                      1.0                  92.677   0       
MHDis


default                             0.49480455220188024  0.422    0       


100%|████████████████████████████████████████████| 1/1 [02:28<00:00, 148.37s/it]

{'gamma': 0.1, 'C': 1}                                                                     1.0                  148.366  0       
      gamma    C        F1    Std      Time    No  Attack
--  -------  ---  --------  -----  --------  ----  --------
 0     1      10  0.999849      0   537.668     0  SYN
 1     1       1  1             0   642.06      0  HTTP
 2     1      10  1             0   207.411     0  ACK
 3     0.1    10  1             0    59.592     0  UDP
 4     0.01    1  1             0  1529.95      0  ARP
 5     1       1  1             0   118.961     0  SP
 6     0.01   10  1             0   101.159     0  BF
 7     0.01   10  1             0   303.606     0  OS
 8     1      10  1             0    92.677     0  SCHD
 9     0.1     1  1             0   148.366     0  MHDis
